In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,16450.45,-0.12,-0.01,0.08,-0.00,-0.01,0.00,-0.00,-0.00
3,-0.12,3961.83,-0.02,0.01,-0.01,0.00,0.01,0.00,0.00
4,-0.01,-0.02,949.29,0.02,-0.00,-0.00,-0.00,-0.00,0.00
5,0.08,0.01,0.02,223.90,0.00,-0.00,0.00,0.00,0.00
6,-0.00,-0.01,-0.00,0.00,51.26,-0.00,-0.00,0.00,0.00
7,-0.01,0.00,-0.00,-0.00,-0.00,11.64,0.00,0.00,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.66,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.63,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,-0.00000,-0.00003,0.00002,-0.00001,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00007,0.00000,0.00002
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00003,1.00000,0.00001,-0.00003,0.00007,0.00001,0.00001
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00001,-0.00000,-0.00003,-0.00001,1.00000,0.00002,0.00000,0.00001
8,0.00002,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00001,0.00000,-0.00001,0.00001,0.00003,0.00000,0.00003,1.00000,-0.00001
10,-0.00001,0.00002,0.00002,0.00001,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.21556527997424887

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.00991586e+07 1.71738525e+06 9.65932903e+04 5.22517886e+03
 2.63112702e+02 1.27385557e+01 5.96594864e-01 2.56909883e-02
 1.20567175e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999464,-0.031902,-0.007171,-0.001680,-0.000386,-0.000088,-0.000020,-0.000005,-0.000001
1,0.031605,0.998800,-0.036491,-0.008066,-0.001845,-0.000417,-0.000094,-0.000023,-0.000005
2,0.008234,0.035861,0.998405,-0.041820,-0.008990,-0.002034,-0.000468,-0.000110,-0.000026
3,0.002252,0.009391,0.040982,0.997875,-0.048568,-0.010376,-0.002358,-0.000557,-0.000133
4,0.000618,0.002570,0.010735,0.047397,0.996917,-0.060097,-0.012911,-0.003028,-0.000724
5,0.000175,0.000724,0.003022,0.012845,0.058223,0.994792,-0.080570,-0.017733,-0.004222
6,0.000052,0.000216,0.000910,0.003842,0.016845,0.076734,0.988851,-0.123477,-0.027209
7,0.000018,0.000073,0.000305,0.001292,0.005644,0.024904,0.111896,0.964657,-0.237197
8,0.000009,0.000037,0.000153,0.000650,0.002840,0.012512,0.054678,0.232091,0.971071


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005362131489313748,
 0.00120025794886236,
 0.0015952060355279851,
 0.0021248361575071373,
 0.003083143535415833,
 0.005207800876915303,
 0.011149390070707477,
 0.035343006888676975,
 0.028928969571111462]